# Projet 1 (semaines 1 à 4) : Reversi 

## Semaine 4 : Reversi Bots, enfin !



In [1]:
%load_ext autoreload
%autoreload 2
from reversi import Reversi, play_game
import tme1
import tme2
import tme4
import numpy as np
import matplotlib.pyplot as plt
from bitreversi import BitboardReversi
from time import time

Dans ce TME, vous allez programmer deux bots de Reversi, les deux basés sur le même principe : l'exploration par Monte-Carlo d'un état du jeu (Monte-Carlo Tree Search, MCTS). Le premier est appelé *Flat Monte-Carlo Tree Search*  ou parfois *Pure Monte-Carlo Tree Search*, le second intègre la variante UCT pour *UCB applied to Trees*.


## Flat MCTS

Vu la grande complexité de l'exploration de l'arbre des possibilités pour un état du jeu, l'idée du Flat MCTS est d'échantilloner au hasard un certain nombre de simulations à chaque coup que l'on doit jouer. Le coup choisi au final est celui qui a le meilleur rendement (la plus grande proportion de victoire). L'algorithme  permet de réaliser un très grande nombre de simulations par coup, vu qu'un agent aléatoire est utilisé pour simuler toutes les parties (quasiment aucun coût computationnel). C'est également un désavantage, vu que le rendement d'un coup est estimé par un agent qui joue aléatoirement (on peut bien sûr améliorer l'agent en intégrant un certain nombre d'heuristiques, mais en augmentant ainsi le coût computationnel).

Le module **reversi.py** tel quel est trop lent pour pouvoir réaliser un grand nombre de simulations. Téléchargez le module **bitreversi.py** qui réimplémente le jeu en version **bitboard** (avec exactement les mêmes méthodes). Les lignes suivantes permettent de comparer la vitesse pour 10000 simulations.

Codez dans la suite un bot **AgentFlatMCTS(board,nb_simu)** qui pour chaque coup à jouer, réalise **nb_simu** joués complètement au hasard et renvoie le coup avec le meilleur ratio de victoires. Vous pouvez utiliser pour cela  **simu_mc** codée en TME 2. Comparez en faisant jouer votre agent contre l'agent aléatoire du TME 1.

In [2]:
game_sans_bitboard = Reversi()
game = BitboardReversi()
t0 = time()
game_sans_bitboard.reset()
tme2.simu_mc(game_sans_bitboard,1,1000) #10000)
print("Sans bitboard : ",time()-t0)

t0 = time()
game.reset()
tme2.simu_mc(game,1, 1000) #10000)
print("Avec bitboard : ",time()-t0)


Sans bitboard :  18.480770111083984
Avec bitboard :  2.8916845321655273


In [3]:
agent_random = tme1.AgentRandom(game)
agent_flat = tme4.AgentFlatMCTS(game,nb_simu=100)

In [4]:
res=[]
t0=time()
for _ in range(5): 
    res.append(play_game(game,agent_random,agent_flat))
print("Résultats : ",res ," Temps : ",time()-t0)

Résultats :  [-32, -20, -18, -18, -32]  Temps :  112.36919355392456


## Monte-Carlo avec UCT

UCT désigne l’algorithme UCB adapté aux arbres de jeu (communément appelé également
Monte Carlo Tree Search). Contrairement à l’algorithme précédent qui explore complétement aléatoirement l’arbre des possibilités, l’idée de l’algorithme est d’explorer en priorité les actions qui ont le plus d’espoir d’amener à une victoire, tout en continuant à explorer d’autres solutions possibles. On retrouve le dilemme d’exploration/exploitation : vaut-il mieux continuer à lancer des simulations aléatoires sur une action qui est pour l’instant la plus performante afin de s’assurer de sa qualité, ou vaut-il mieux explorer d’autres actions possibles ? Pour cela, le principe est d’utiliser l’algorithme UCB à chaque embranchement possible, afin d’équilibrer l’exploration et l’exploitation. Vous trouverez une illustration explicative sur [la page wikipedia](https://en.wikipedia.org/wiki/Monte_Carlo_tree_search).

La racine correspond à l’état courant du jeu. Comme dans le cas de l’algorithme de Monte-Carlo, on a besoin d’une stratégie par défaut pour explorer rapidement la qualité d’un état (la probabilité de gagner à partir de cet état). Cette stratégie par défaut est la stratégie aléatoire. Au tout début, aucune information n’est disponible, on va simuler pour chaque action une partie avec le joueur aléatoire pour initialiser les nœuds enfants de la racine. Seuls ces nœuds enfants, correspondant chacun à une action, sont pour l’instant créés ; les états visités lors de la simulation du jeu ne sont pas stockés ! Chaque nœud ainsi développé
stockera le nombre de fois où ce nœud à mener à une victoire et le nombre de fois où il a été joué (à participer à une simulation, victoire défaite ou match nul). A partir d’un arbre déjà en partie exploré, les différentes étapes sont les suivantes :

* Sélection : un nœud à explorer est sélectionné dans l’arbre. Pour cela, en partant de la racine, on choisit le nœud suivant en fonction de l’algorithme UCB parmi les enfants du nœud courant, jusqu’à tomber soit sur une feuile de l’arbre (un nœud sans enfant) soit sur un état où une des actions n’a jamais été explorée (pas de nœud fils correspondant à cette action).
* Expansion : une fois le nœud sélectionné, pour une action jamais effectuée à ce nœud, un nœud fils est créé et simulé.
* Simulation : à partir d’un nœud à simuler, un jeu entre deux joueurs aléatoires est déroulé jusqu’à atteindre un état final (victoire, défaite ou match nul). Les états visités lors de la simulation ne sont pas ajoutés à l’arbre, ils ne sont pas stockés.
* On rétro-propage le résultat de la simulation à tout le chemin menant au fils simulé : pour chaque nœud sur le chemin, on met à jour en fonction du résultat le nombre de victoires et le nombre de visites.

Le processus est itéré en fonction du nombre de simulation (ou temps) disponible. A la fin, le coup le plus visité est renvoyé (plutot que le coup avec le meilleur rendement, car il est considéré plus sûr que celui avec le meilleur rendement si le nombre de simulations a été assez grand). 

#### Implémentation

Vous aurez besoin d'une classe **MCTSNode(game,player,parent=None)** pour stocker les informations relatives à un noeud :
* player : à qui le tour de jouer (1 ou -1)
* parent : le noeud parent (le noeud stocke la structure de l'arbre)
* children : un dictionnaire où à un coup est associé le noeud enfant résultant
* config : le résultat de **game.board_to_int()** (pour ne pas stocker le jeu complet, il est possible de reconstruire l'état du jeu en utilisant la méthode **game.bitboards_to_board(*config)**
* visits : nombre de fois où le noeud a été visité
* wins : nombre de victoires associées au noeud
* untried_moves : liste des coups non joués du noeud, initialisée par **game.valid_moves(player)**

et les méthodes : **is_fully_expanded()** qui permet de tester s'il y a encore des coups non explorés (**untried_moves** non vide), et **best_move()** qui renvoie le couple *(coup,noeud fils)* stocké dans le dictionnaire qui maximise la formule UCT : $UCB_{child} = \mu_{child}+\sqrt{K*\frac{log(T)}{T_{child}}}$ avec $\mu_{child}$ le rapport **wins/visits** du noeud fils, $T_{child}$ le nombre de fois que l'on a visité le noeud fils et $T$ le nombre de fois que l'on a visité le noeud courant (et K une constante qui favorise ou non l'exploration).

Vous pouvez ensuite coder la classe **AgentMCTS(game,nb_simu)**, dont la méthode **play(player)** suit la description précédente :
* si le noeud courant n'a jamais été visité (vous pouvez conserver les noeuds visités dans un dictionnaire), un nouveau **MCTSNode** est créé, sinon on récupère celui qui existe. Puis, pour **nb_simu**, on initialise le noeud à l'état courant. 
- Sélection : tant que toutes les actions du noeud courant ont été explorées (**is_fully_expanded** à vrai), et tant que ce n'est pas la fin du jeu, on sélectionne le noeud suivant avec la fonction * **best_move** du noeud;
- Expansion : le noeud courant contient au moins un coup non exploré, on en choisit un et on l'enlève de **untried_moves**; le coup est joué et un nouveau noeud est ajouté (sans oublier de préciser  son parent). - Simulation : une partie complète (avec la fonction **rollout** du TME 2) est jouée.
- Rétro-propagation : en fonction du résultat (attention à bien inversé le résultat si c'était au joueur 2 de jouer), on remonte depuis le noeud final de parent en parent en mettant à jour les variables **wins** et **visits** des noeuds. 
* Le coup renvoyé est celui qui a été le plus visité à partir du noeud correspondant à l'état courant. 


Testez votre agent contre le random et le Pure Monte-Carlo. Que remarquez vous pour le temps d'exécution des trois agents ? Est-ce fair-play ? Comment assurer l'équité ? Est-ce que K joue un rôle ?

In [27]:
agent_uct = tme4.AgentMCTS(game)

In [28]:
res=[]
t0=time()
for _ in range(5): 
    res.append(play_game(game,agent_random,agent_uct))
print("Résultats : ",res ," Temps : ",time()-t0)

Résultats :  [22, 42, 20, 32, 8]  Temps :  167.61306858062744


In [29]:
res=[]
t0=time()
for _ in range(5): 
    res.append(play_game(game,agent_flat,agent_uct))
print("Résultats : ",res ," Temps : ",time()-t0)

Résultats :  [39, 18, 43, 20, 34]  Temps :  277.6788659095764


## Améliorations possibles

Vous pouvez tout à fait hybrider la version flat ou la version UCT avec des heuristiques pour guider plus vite UCB vers les zones intéressantes, et ce de plusieurs manières, entre autres : 

* Si on dispose d'une fonction de score heuristique $h$ pour un état, le score UCB d'un noeud $child$ peut être modifié par addition ou multiplication $UCB_{child}+\alpha*h(child)$ ou $UCB_{child}*(1+\alpha h(child))$
* On peut également biaiser l'estimation initiale de la moyenne empirique du rendement des noeuds $\hat{\mu}_{child} = \frac{\mu_{child}+\beta h(child)}{T_{child}+\beta}$.
* le terme exploratoire peut également être modulé : $UCB_{child} = \mu_{child} + \sqrt{K*\frac{log(T)}{T_{child}*f(child)}}$


## Tournoi de bots

Pour clôturer cette série de TMEs, un tournoi de bots sera organisé. 
Vous soumettrez votre code sur le lien suivant : [https://nuage.isir.upmc.fr/index.php/s/yrAdksZDa5kB7mZ](https://nuage.isir.upmc.fr/index.php/s/yrAdksZDa5kB7mZ) et il devra **obligatoirement suivre les spécifications suivantes** : 
* un fichier **Agent_XXXX_YYYY.zip** qui contiendra un répertoire de la forme **Agent_XXXX_YYYY** avec XXXX et YYYY les numéros d'étudiants du binome (si monome, juste XXXX). 
* dans ce répertoire un fichier **agent.py** qui contiendra une classe dont le nom commence par **Agent**. Cet agent devra avoir forcément **un constructeur** qui prend en paramère un objet de type **BitboardReversi**, une variable d'instance **board** qui stocke le jeu et une méthode **play(player)** qui renvoie un couple d'entiers, le coup à jouer. Vous pouvez placer d'autres fichiers/modules si nécessaires dans le repertoire. Attention par contre à n'avoir qu'une seule classe commençant par **Agent** ! (sinon la classe sera choisie aléatoirement...).
* votre agent aura 2 secondes par coup pour jouer.

Les agents sont à soumettre au plus tard la veille du TME 5.

Le script pour jouer le tournoi est disponible sur le moodle (**main_reversi.zip**), vous pouvez vérifier que votre code marche en plaçant votre répertoire **AGENT_XXXX_YYYY** dans le répertoire **agents** et en éxecutant le script **main_reversi.py**.

Vous trouverez dans le zip un exemple d'agent aléatoire.

Vous pouvez indiqué dans un fichier README.txt  les améliorations que vous avez effectuées (si jamais vous avez eu le temps d'en faire). 

